In [19]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification,AutoTokenizer, AutoModelForSequenceClassification
import torch

In [20]:
# pip install transformers

In [21]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

text = "I really dislike this item."

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Move inputs to the correct device
inputs = {key: value.to(device) for key, value in inputs.items()}

# Get model predictions
with torch.no_grad():
    outputs = model(**inputs)

In [23]:
# Get the predicted class (0 for negative, 1 for positive)
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

# Display the result
sentiment = "positive" if predicted_class == 1 else "negative"
print(f"The sentiment is: {sentiment}")

The sentiment is: negative


In [24]:
model.framework

'pt'

In [25]:
from patra_model_card.patra_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

In [26]:
mc = ModelCard(
    name="Sentiment Analysis Model using Hugging Face",
    version="0.1",
    short_description="Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on SST-2 dataset.",
    full_description="This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the SST-2 dataset. It is capable of classifying text as either positive or negative sentiment.",
    keywords="sentiment analysis, hugging face, transformers, distilbert, patra",
    author="Isuru Gamage",
    input_type="Text",
    category="classification",
    foundational_model="None",
)

mc.input_data = 'https://huggingface.co/datasets/sst2'
mc.output_data = 'https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english'

In [27]:
ai_model = AIModel(
    name="Sentiment Analysis Hugging Face Model",
    version="1.0",
    description="Sentiment analysis model using Hugging Face Transformers pre-trained DistilBERT model for positive and negative sentiment classification.",
    owner="Isuru Gamage",
    location="https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english", 
    license="Apache-2.0", 
    framework="pytorch",
    model_type="other",  
    test_accuracy=0  
)


ai_model.populate_model_structure(model)

In [28]:
mc.ai_model = ai_model

In [29]:
mc.populate_requirements()

In [30]:
print(mc)

{
    "name": "Sentiment Analysis Model using Hugging Face",
    "version": "0.1",
    "short_description": "Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on SST-2 dataset.",
    "full_description": "This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the SST-2 dataset. It is capable of classifying text as either positive or negative sentiment.",
    "keywords": "sentiment analysis, hugging face, transformers, distilbert, patra",
    "author": "Isuru Gamage",
    "input_type": "Text",
    "category": "classification",
    "input_data": "https://huggingface.co/datasets/sst2",
    "output_data": "https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english",
    "foundational_model": "None",
    "ai_model": {
        "name": "Sentiment Analysis Hugging Face Model",
        "version": "1.0",
        "description": "Sentiment analysis model using Hugging Face Transformers pre-trained 

In [31]:
mc.validate()

True

In [32]:
mc.submit("http://127.0.0.1:5002/upload_mc")

{'message': 'Successfully uploaded the model card',
 'model_card_id': 'ee74c7d3-af30-465b-8a23-d2f232c648d3'}

Retrain the HaggingFace Model

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

In [34]:

df = pd.read_csv("/Users/agamage/Desktop/D2I/Codes/fork/patra-toolkit/examples/notebooks/data/sentiment/news_sentiment_analysis.csv")  # Adjust the path as necessary

df = df.drop(columns=['URL', 'Source','Author'])
# Combine title and description as features
df['Text'] = df['Title'] + " " + df['Description']

label_encoder_type = LabelEncoder()
label_encoder_date = LabelEncoder()

df['Type'] = label_encoder_type.fit_transform(df['Type'])

df['Published_day'] = pd.to_datetime(df['Published At']).dt.day_name()
df['Published At'] = label_encoder_date.fit_transform(df['Published_day'])

df = df.drop(columns=['Title', 'Description', 'Published_day'])

# Map sentiment labels from -1 (negative) to 0, and keep 1 as positive
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == "positive" else 0)

# Split the data into features (X) and labels (y)
X = df.drop(columns=['Sentiment'])
y = df['Sentiment'] # Target label

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)



device = torch.device("mps") if torch.has_mps else (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
model.to(device)

# Tokenize the training and test data
train_encodings = tokenizer(X_train['Text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer(X_test['Text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Move input tensors to the correct device
train_encodings = {key: val.to(device) for key, val in train_encodings.items()}
test_encodings = {key: val.to(device) for key, val in test_encodings.items()}

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train.tolist()).to(device)
y_test_tensor = torch.tensor(y_test.tolist()).to(device)


/var/folders/hz/704s19kd1zg3phgs0khv5df00000gp/T/ipykernel_66489/2829974262.py:11: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps") if torch.has_mps else (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))


In [36]:
# Create a custom Dataset class

class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewsDataset(train_encodings, y_train_tensor)
test_dataset = ReviewsDataset(test_encodings, y_test_tensor)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Number of training epochs
    per_device_train_batch_size=16,  # Batch size for training
    evaluation_strategy="epoch",      # Evaluate at the end of each epoch
    logging_dir='./logs',            # Directory for storing logs
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


evaluation_results = trainer.evaluate() 

print(evaluation_results)

model.eval()

/Users/agamage/Desktop/patra-toolkit/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.361191
2,No log,0.394380
3,0.236700,0.473072


{'eval_loss': 0.4730719029903412, 'eval_runtime': 7.6498, 'eval_samples_per_second': 91.505, 'eval_steps_per_second': 11.504, 'epoch': 3.0}


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [37]:
# pip install 'accelerate>={ACCELERATE_MIN_VERSION}'

In [38]:
with torch.no_grad():
    test_outputs = model(**test_encodings)

In [39]:

device = torch.device("mps") if torch.has_mps else torch.device("cpu")

new_reviews = ["This product is amazing!", "I really dislike this item."]

# Tokenize the new reviews
encodings = tokenizer(new_reviews, truncation=True, padding=True, max_length=512, return_tensors="pt")

encodings = {key: val.to(device) for key, val in encodings.items()}

model = model.to(device)

# predictions
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encodings)

# predicted class (0 for negative, 1 for positive)
predictions = torch.argmax(outputs.logits, dim=-1)

for review, prediction in zip(new_reviews, predictions):
    sentiment = "Positive" if prediction == 1 else "Negative"
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

/var/folders/hz/704s19kd1zg3phgs0khv5df00000gp/T/ipykernel_66489/2963717503.py:2: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps") if torch.has_mps else torch.device("cpu")


Review: This product is amazing!
Predicted Sentiment: Positive

Review: I really dislike this item.
Predicted Sentiment: Negative



In [40]:
from patra_model_card.patra_model_card import ModelCard, AIModel, BiasAnalysis, ExplainabilityAnalysis

In [41]:
mct = ModelCard(
    name="Retrain Sentiment Analysis Model using Hugging Face and News dataset",
    version="0.1",
    short_description="Retrain Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on News dataset.",
    full_description="This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the News dataset. It is capable of classifying text as either positive or negative sentiment.",
    keywords="sentiment analysis, hugging face, transformers, distilbert, patra, retrain",
    author="Isuru Gamage",
    input_type="Text",
    category="classification",
    foundational_model="c231bbd8-b934-4e96-a244-3dcf4d8dd344",
)

# Link to input and output data (in the context of Hugging Face's pre-trained model and dataset)
mct.input_data = 'https://huggingface.co/datasets/news'
mct.output_data = 'https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english'

In [42]:
ai_model = AIModel(
    name="Retrain Sentiment Analysis Hugging Face Model",
    version="1.0",
    description="Retrain Sentiment analysis model using Hugging Face Transformers pre-trained DistilBERT model for positive and negative sentiment classification.",
    owner="Isuru Gamage",
    location="https://huggingface.co/distilbert-base-uncased-finetuned-news-english/train", 
    license="Apache-2.0", 
    framework="pytorch",
    model_type="other",  
    test_accuracy=0  
)


ai_model.populate_model_structure(model)

In [43]:
ai_model.add_metric("Test loss", 0.22)
ai_model.add_metric("Epochs", 100)
ai_model.add_metric("Batch Size", 32)
ai_model.add_metric("Optimizer", "Adam")
ai_model.add_metric("Learning Rate", 0.0001)
ai_model.add_metric("Input Shape", "(26048, 100)")

In [44]:
mct.ai_model = ai_model

In [45]:
mct.populate_requirements()

In [46]:
from torch.nn.functional import softmax
import numpy as np

y_pred_prob = softmax(test_outputs.logits, dim=1).cpu().numpy()

print(f"Probabilities for the first 5 samples: \n{y_pred_prob[:5]}")

# If you want to convert these probabilities into binary predictions:
y_pred = np.argmax(y_pred_prob, axis=1)

Probabilities for the first 5 samples: 
[[7.3398073e-04 9.9926597e-01]
 [6.1090710e-03 9.9389088e-01]
 [9.9954009e-01 4.5991005e-04]
 [1.7833044e-03 9.9821669e-01]
 [1.1440295e-03 9.9885595e-01]]


In [48]:
print(mct)

{
    "name": "Retrain Sentiment Analysis Model using Hugging Face and News dataset",
    "version": "0.1",
    "short_description": "Retrain Sentiment analysis model based on Hugging Face's DistilBERT fine-tuned on News dataset.",
    "full_description": "This model utilizes the Hugging Face Transformers framework with the DistilBERT model, fine-tuned for sentiment analysis on the News dataset. It is capable of classifying text as either positive or negative sentiment.",
    "keywords": "sentiment analysis, hugging face, transformers, distilbert, patra, retrain",
    "author": "Isuru Gamage",
    "input_type": "Text",
    "category": "classification",
    "input_data": "https://huggingface.co/datasets/news",
    "output_data": "https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english",
    "foundational_model": "https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english",
    "ai_model": {
        "name": "Retrain Sentiment Analysis Hugging Face Model",
     

In [49]:
mct.validate()

True

In [50]:
mct.submit("http://127.0.0.1:5002/upload_mc")

{'message': 'Successfully uploaded the model card',
 'model_card_id': 'ad5123d0-0a23-4803-9c99-3ec66afb85d0'}